In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def load_data(path):
    data = np.array([])
    label = np.array([])
    data = np.zeros((31000,50,30))
    label = np.zeros(31000)
    k = 0
    l = 0
    for i in tqdm(range(0,62)):
        for j in range(0,500):
            img = cv.imread(str(path) + f'/{i}/{j}.jpg')
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            img[img<=115]=0
            data[k] = np.array(img).reshape(50,30)
            label[k] = l
            k += 1
        l += 1
    return (data,label)

In [4]:
imgs, labels = load_data('Data')

100%|███████████████████████████████████████████| 62/62 [00:04<00:00, 14.21it/s]


In [6]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(50,30,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(62))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 28, 16)        160       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 14, 32)        4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 7, 32)        0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 12, 7, 32)        128       
 hNormalization)                                                 
                                                      

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(imgs, labels, epochs=10)

Epoch 1/10
969/969 [==============================] - 24s 23ms/step - loss: 0.6765 - accuracy: 0.8185
Epoch 2/10
969/969 [==============================] - 22s 22ms/step - loss: 0.0362 - accuracy: 0.9894
Epoch 3/10
969/969 [==============================] - 21s 22ms/step - loss: 0.0385 - accuracy: 0.9883
Epoch 4/10
969/969 [==============================] - 21s 22ms/step - loss: 0.0317 - accuracy: 0.9912
Epoch 5/10
969/969 [==============================] - 21s 22ms/step - loss: 0.0255 - accuracy: 0.9929
Epoch 6/10
969/969 [==============================] - 21s 22ms/step - loss: 0.0068 - accuracy: 0.9979
Epoch 7/10
969/969 [==============================] - 21s 21ms/step - loss: 0.0026 - accuracy: 0.9992
Epoch 8/10
969/969 [==============================] - 21s 21ms/step - loss: 0.0326 - accuracy: 0.9917
Epoch 9/10
969/969 [==============================] - 21s 22ms/step - loss: 0.0048 - accuracy: 0.9986
Epoch 10/10
969/969 [==============================] - 21s 22ms/step - loss: 0.002

In [11]:
model.save("Model/captcha.h5")